In [12]:
import pandas as pd

# Пункт 2
"""
Буду считать, что продажа-покупка в одну и ту же секунду невозможна.
Так как датасет имеет более 260000 значений, бесполезно хранить все точки для поиска максимальной
прибыли за 2 транзакции. Именно поэтому будем рассматривать лишь экстремальные точки.
Основная стратегия - поиск локальных экстремумов и нахождение разницы между ними.
Если замечаем, что по обе стороны функция выше нашего значения - локальный минимум
Если замечаем, что по обе стороны функция ниже нашего значения - локальный максимум
Если замечаем, что по одну сторону функция выше нашего значения, а по другую равна - локальный минимум
Если замечаем, что по одну сторону функция ниже нашего значения, а по другую равна - локальный максимум
Очевидно, что максимальная прибыль будет в двух операциях, таких что первая купля-продажа будет начинаться с глобального минимума, а вторая купля-продажа
будет заканчиваться глобальным максимумом.
Отмечу, что я мог пользоваться преимуществами библиотек pandas и numpy, но попытался свести их к минимуму, чтобы продемонстрировать владение языком без
сторонних модулей.
"""

# Считаем файл
df = pd.read_csv('new.csv')


class Bank_account:
    def __init__(self):
        # Контейнер отвечает за инфу(дата, время, прайс, операция)
        # Будем считать прибыль с помощью 2ой перменной
        self._bank_account_info, self._bank_account_profit = [], 0

    # Покупка акций
    def buy(self, price_now, time, date):
        self._bank_account_info.append([price_now,
                                        time,
                                        date,
                                        'buy'])

    # Продажа акций
    def sell(self, price_now, time, date):
        # Так как мы всегда покупаем перед продажей, вычитаем нашу сумму продажи из предыдущей операции(которая всегда покупка)
        self._bank_account_profit += price_now - self._bank_account_info[-1][0]
        # Добавляем инфу
        self._bank_account_info.append([price_now,
                                        time,
                                        date,
                                        'sell'])

    # Вывод информации
    def get_info(self):
        for elements in self._bank_account_info:
            print(elements)
        print('Изменение стоимости акций:',
              self._bank_account_profit)


# Функция поиска прибыли между максимумами и глобальным минимумов, а также между минимумами и глобальным максимумом:
def search():
    # Массивы минимумов и максимумов соотвественно
    mins, maxs = [], []
    # Нам нужен массив данных со стоимостью, временем и датой(получаем np.array([]))
    data = df[['price', 'time', 'date']].values
    # Добавим начальную точку датасета в максимум(заметил это из графика, который построил)
    maxs.append([data[-1][0], data[-1][1], data[-1][2]])
    # Добавим конечную точку датасета в минимум(заметил это из графика, который построил)
    mins.append([data[0][0], data[0][1], data[0][2]])
    # Идем по циклу и смотрим соседние точки, нашли, добаляем точку как максимум/минимум + инфу о времени и дате
    for i in range(1, len(data) - 1):
        if ((data[i][0] < data[i + 1][0] and data[i][0] < data[i - 1][0])
                or (data[i][0] < data[i + 1][0] and data[i][0] == data[i - 1][0])
                or (data[i][0] == data[i + 1][0] and data[i][0] < data[i - 1][0])):
            mins.append([data[i][0], data[i][1], data[i][2]])
        elif ((data[i][0] > data[i + 1][0] and data[i][0] > data[i - 1][0])
              or (data[i][0] > data[i + 1][0] and data[i][0] == data[i - 1][0])
              or (data[i][0] == data[i + 1][0] and data[i][0] > data[i - 1][0])):
            maxs.append([data[i][0], data[i][1], data[i][2]])
    # Инициализируем словарь расстояний между max и min
    # Сам по себе он будет нести мало информации, поэтому привяжем к нему max, min, max_time и min_time
    destinations = {}
    """
    В первом цикле смотрим на разницу между всеми максимумами и глобальным минимумом.
    Очевидно, что сначала проверяем меньше ли global_min_time, чем max_time(потому что мы должны купить в минимуме, а продать в максимуме).
    Затем, если условие выполняется, проверяем есть ли уже такая прибыль(прибыль может быть составлена различными комбинациями).
    Аналогично во втором цикле, только со всеми минимумами и глобальным максимумом.
    """
    global_min = min(mins)
    global_max = max(maxs)
    for i in range(len(maxs)):
        if global_min[1] < maxs[i][1]:
            # Если уже есть разница, добавляем другую комбинацию min, max, из которой можно получить такую же прибыль
            if maxs[i][0] - global_min[0] in destinations.keys():
                destinations[maxs[i][0] - global_min[0]].append([maxs[i][0],
                                                                 global_min[0],
                                                                 maxs[i][1],
                                                                 global_min[1],
                                                                 maxs[i][2],
                                                                 global_min[2]])
            else:
                destinations[maxs[i][0] - global_min[0]] = [[maxs[i][0],
                                                             global_min[0],
                                                             maxs[i][1],
                                                             global_min[1],
                                                             maxs[i][2],
                                                             global_min[2]]]
    for i in range(len(mins)):
        if global_max[1] > mins[i][1]:
            if global_max[0] - mins[i][0] in destinations.keys():
                destinations[global_max[0] - mins[i][0]].append([global_max[0],
                                                                 mins[i][0],
                                                                 global_max[1],
                                                                 mins[i][1],
                                                                 global_max[2],
                                                                 mins[i][2]])
            else:
                destinations[global_max[0] - mins[i][0]] = [[global_max[0],
                                                             mins[i][0],
                                                             global_max[1],
                                                             mins[i][1],
                                                             global_max[2],
                                                             mins[i][2]]]
    # Сортируем словарь прибылей по прибылям.
    return sorted(destinations.items(), key=lambda x: (-x[0]))


if __name__ == '__main__':
    # Заводим аккаунт
    bank = Bank_account()
    profits = search()
    # Нас интересует максимальная прибыль за 2 шага, поэтому заводим максимум
    maxim = -1
    # Из стратегии, очевидно, что первая операция начинается с покупки в минимуме, а вторая заканчивается продажей в максимуме.
    # Но в массиве profits нет никаких пометок, где глоб. минимум, а где глоб. максимум, поэтому в зависимости от временных соотвестветствий
    # и максимальной прибыли, они могут идти в разном порядке, для этого и заведем массивы first и second, чтобы расставить порядок купли-продажи
    first, second = [], []
    for i in range(len(profits)):
        for j in range(len(profits)):
            # Этим условием откинем множество лишних проверок :)
            if profits[i][0] + profits[j][0] > maxim and i != j:
                """
                А вот этот цикл в цикле тут не просто так, дело в том, что прибыль может получаться различными комбинациями прайсов.
                Например, на первых 50 значениях, мы наблюдаем такое: (16.0, [[1067.0, 1051.0, 105000.0, 102800.0, 20150105.0, 20150105.0], 
                                                                                [1067.0, 1051.0, 104600.0, 102800.0, 20150105.0, 20150105.0], 
                                                                                [1067.0, 1051.0, 105000.0, 102800.0, 20150105.0, 20150105.0]])
                Таким образом, нам нужно проверить все значения из прибыли, чтобы понять, какие будут в сумме с другой прибылью давать максимум,
                при этом не пересекаясь по времени.
                """
                for k in range(len(profits[i][1])):
                    for l in range(len(profits[j][1])):
                        if (profits[j][1][l][2] < profits[i][1][k][3] and
                                profits[j][1][l][3] < profits[i][1][k][3]):
                            maxim = profits[i][0] + profits[j][0]
                            first = [profits[j][1][l][0], profits[j][1][l][1],
                                     profits[j][1][l][2], profits[j][1][l][3],
                                     profits[j][1][l][4], profits[j][1][l][5]]
                            second = [profits[i][1][k][0], profits[i][1][k][1],
                                      profits[i][1][k][2], profits[i][1][k][3],
                                      profits[i][1][k][4], profits[i][1][k][5]]
                        elif (profits[i][1][k][3] < profits[j][1][l][3] and
                              profits[i][1][k][2] < profits[j][1][l][3]):
                            maxim = profits[i][0] + profits[j][0]
                            first = [profits[i][1][k][0], profits[i][1][k][1],
                                     profits[i][1][k][2], profits[i][1][k][3],
                                     profits[i][1][k][4], profits[i][1][k][5]]
                            second = [profits[j][1][l][0], profits[j][1][l][1],
                                      profits[j][1][l][2], profits[j][1][l][3],
                                      profits[j][1][l][4], profits[j][1][l][5]]
    print(first, second)
    bank.buy(first[1], first[3], first[5])
    bank.sell(first[0], first[2], first[4])
    bank.buy(second[1], second[3], second[5])
    bank.sell(second[0], second[2], second[4])
    bank.get_info()

[[1066.0, 104400.0, 20150105.0]] [[1056.0, 104200.0, 20150105.0]]
[(10.0, [[1066.0, 1056.0, 104400.0, 104200.0, 20150105.0, 20150105.0], [1066.0, 1056.0, 104400.0, 104200.0, 20150105.0, 20150105.0]])]
[] []


IndexError: list index out of range